In [1]:
import requests
import pandas as pd
import json
import plotly.express as px
import plotly.graph_objects as go

pd.options.display.float_format = '{:,.2f}'.format

In [2]:
%%capture
requests.post('https://maker.ifttt.com/trigger/heroku_app/with/key/c1rbTcV6gKEl94vAuT3vah', params={'value1':'USA Spending'})

In [3]:
agency_req = requests.get('https://api.usaspending.gov/api/v2/references/toptier_agencies')
agency_json = agency_req.json()['results']

In [4]:
df = pd.DataFrame.from_dict(agency_json)
#df.obligated_amount = df.obligated_amount.astype(float)
#df = df.sort_values('obligated_amount', ignore_index=True)
#df.head(len(df))

In [5]:
agency_ids = df['agency_id'].to_list()
agency_names = df['agency_name'].to_list()

In [6]:
data = []
year=2021
for agency,agency_name in zip(agency_ids,agency_names):
    request = requests.get('https://api.usaspending.gov/api/v2/federal_obligations/?fiscal_year='+str(year)+'&funding_agency_id='+str(agency)+'&limit=500&page=1')
    req_json = request.json()['results']
    for element in req_json:
        element['agency_id'] = agency
        element['agency_name'] = agency_name
        data.append(element)

In [7]:
output = pd.DataFrame.from_dict(data)
output.obligated_amount = output.obligated_amount.astype(float)
#output = output.sort_values('obligated_amount', ignore_index=True)
#output.head(len(output))

In [8]:
output['budget'] = 'Budget'
fig = px.treemap(output, path=['budget', 'agency_name', 'account_title'], values='obligated_amount')
fig.data[0].textinfo = 'label+percent parent+percent root'
fig.data[0].hovertemplate='<b>%{label} </b> <br> Budget: $%{value:,.2f}<br> Percent Parent: %{percentParent:.2%}'
widget = go.FigureWidget(fig)
widget.layout.title = 'USA Budgets in '+str(year)
widget.update_layout(
    autosize=True,
    width=1470,
    height=655,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=50,
        pad=1
    ),
    title_x=0.5
)
widget

FigureWidget({
    'data': [{'branchvalues': 'total',
              'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0…